# ⚙️ Ejercicio 03 – Uso de Oracle Generative AI (LLM) con SDK de Python

## 🎯 Objetivo del laboratorio

Este laboratorio muestra cómo **ejecutar solicitudes a modelos de lenguaje (LLM)** disponibles en **Oracle Cloud Infrastructure (OCI)** usando el **SDK de Python** y autenticación mediante **Resource Principal** o **archivo de configuración local** (`~/.oci/config`).

A través de un ejemplo práctico, aprenderás a:

* Configurar correctamente el entorno local y las credenciales del SDK.
* Crear y verificar la conexión con el servicio **Generative AI Inference**.
* Construir una solicitud (*chat request*) tipo conversación con texto.
* Invocar un modelo de lenguaje y obtener una respuesta en formato JSON.

El objetivo final es entender cómo el SDK gestiona los parámetros del modelo (tokens, temperatura, `top_p`, etc.) y cómo integrar esta llamada dentro de flujos de inferencia, notebooks o pipelines en OCI Data Science.

---

## 🧩 Contexto

Oracle ofrece una serie de modelos **Generative AI** accesibles por API, compatibles con chat, summarization, embeddings, clasificación, entre otros.

En este laboratorio se usa un **modelo de tipo “CHAT”**, identificado mediante su **OCID** (`MODEL_ID`), desplegado en la región `us-chicago-1`. El flujo emplea el cliente `GenerativeAiInferenceClient` del SDK `oci`, que permite conectarse directamente al endpoint:

```python
SERVICE_ENDPOINT = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
```

---

## ⚙️ Pasos principales

1. **Configuración de autenticación**

   * Crear la carpeta `~/.oci/` y colocar el archivo `config` con los parámetros: `user`, `tenancy`, `region`, `fingerprint`, `key_file`. # Si ya hiciste el ejercicio 01 omite el primer punto)
 
2. **Inicialización del cliente**

   * Importar y configurar `GenerativeAiInferenceClient` con el `signer` y el `SERVICE_ENDPOINT`.

3. **Creación de la solicitud**

   * Construir un objeto `GenericChatRequest` con el texto del usuario, los parámetros del modelo (`temperature`, `max_tokens`, `top_p`, etc.), y el rol `"USER"`.

4. **Ejecución e interpretación de la respuesta**

   * Enviar la solicitud mediante `inf.chat(chat_detail)` y visualizar la respuesta formateada en JSON.

---

## 🧠 Ejemplo de salida

```json
{
  "response": "\"Llama\" puede tener diferentes significados dependiendo del contexto..."
}
```

El modelo interpreta el prompt textual y devuelve una respuesta estructurada con contexto, demostrando el flujo completo de **pregunta → inferencia → respuesta generativa**.

---



In [ ]:
!oci --version

## 01 - Configuración de la autenticación del SDK de OCI ( si ya ejecutaste el ejercicio 01, no es necesario ejecutar estos pasos, puedes pasar al paso 2)

In [ ]:
# crea carpeta y permisos
#!mkdir -p /home/datascience/.oci

In [ ]:
# Ver tu HOME y listar (incluye ocultos)
#!echo $HOME
#!ls -la $HOME | head -n 30

In [ ]:
#!mkdir -p ~/.oci
#!ls -la ~/.oci

In [ ]:
# %%bash
# cat > ~/.oci/config <<'CFG'
# [DEFAULT]
# user=ocid1.user.oc1..
# tenancy=ocid1.tenancy.oc1..a
# region=us-chicago-1
# fingerprint=6d:a2:b6:25:24:40:11:
# key_file=/home/datascience/.oci/oci_api_key.pem
# CFG

# echo "Config creado en ~/.oci/config"
# cat ~/.oci/config | sed 's/fingerprint=.*/fingerprint=<oculto>/'

In [ ]:
# Quita posibles finales de línea de Windows (CRLF)
# !sed -i 's/\r$//' ~/.oci/config

# # mostrar
# !sed -n '1,200p' ~/.oci/config

# 2. Usando los Modelos de LLM usando Python

In [ ]:
user_input = "qué es llama?"

In [ ]:
import oci
import json
from oci.auth.signers import get_resource_principals_signer

In [ ]:
# === Config ===
REGION = "us-chicago-1"
SERVICE_ENDPOINT = f"https://inference.generativeai.{REGION}.oci.oraclecloud.com" #NO MODIFICAR
COMPARTMENT_ID = "ocid1.compartment.oc1...." #PARAMETRO A ACTUALIZAR
MODEL_ID = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceya6dvgvvj3ovy4lerdl6fvx525x3yweacnrgn4ryfwwcoq" #NO MODIFICAR

In [ ]:

# === Signer===
signer = get_resource_principals_signer()
cfg = {"region": REGION}

# (opcional)
if MODEL_ID is None:
    from oci.generative_ai import GenerativeAiClient
    genai = GenerativeAiClient(config=cfg, signer=signer)
    models = genai.list_models(
        compartment_id=COMPARTMENT_ID,
        capability=["CHAT"],
        lifecycle_state="ACTIVE"
    ).data.items
    assert models, "No hay modelos CHAT visibles en el compartimento. Revisa permisos/compartimento."
    MODEL_ID = models[0].id
    print("Usando modelo:", MODEL_ID)

# === Cliente de inferencia ===
inf = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=cfg, signer=signer, service_endpoint=SERVICE_ENDPOINT
)

# === Prompt del usuario ===
user_input = user_input

# --- Construcción del request ---
content = oci.generative_ai_inference.models.TextContent(text=user_input)
message = oci.generative_ai_inference.models.Message(role="USER", content=[content])

chat_request = oci.generative_ai_inference.models.GenericChatRequest(
    api_format=oci.generative_ai_inference.models.BaseChatRequest.API_FORMAT_GENERIC,
    messages=[message],
    max_tokens=600,
    temperature=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    top_p=0.75,
)

chat_detail = oci.generative_ai_inference.models.ChatDetails(
    serving_mode=oci.generative_ai_inference.models.OnDemandServingMode(model_id=MODEL_ID),
    chat_request=chat_request,
    compartment_id=COMPARTMENT_ID,
)



In [ ]:
# === Llamada ===
resp = inf.chat(chat_detail)

# === Resultado ===
choices = resp.data.chat_response.choices
response_text = choices[0].message.content[0].text if choices else "No se generó respuesta."
print(json.dumps({"response": response_text}, indent=2, ensure_ascii=False))